In [7]:
import pandas as pd

# TODO: Split this into subfunctions if necessary
# TODO: Create additional unit tests for robustness

#A function that takes in a csv of student names, emails, team names, and returns team/teammatenumbers/teammate names for Qualtrics
def createRosterForQualtrics(csvFileStr, firstNameCol, lastNameCol, teamNameCol, hasTeamNumbers, outputFileName):
    
    #importing and sorting data
    raw = pd.read_csv(csvFileStr)
    raw["FullName"] = raw[firstNameCol] + ' ' + raw[lastNameCol]
    raw["FullNameUpper"] = raw["FullName"].str.upper()
    raw = raw.sort_values(by=[teamNameCol, "FullNameUpper"], ascending=True)
    teams = raw[teamNameCol].unique()
    
    #instantiate new columns to add
    teammatenumbers = []
    teamnumbers = []
    team_sizes = []
    
    #figure out max team size
    for team in teams:
        subset_team = raw[raw[teamNameCol] == team]
        team_sizes.append(len(subset_team))
    max_team_size = max(team_sizes)
    
    #dynamically instantiate teammate columns
    teammate_cols = {}
    for i in range(max_team_size):
        teammate_str = "Teammate" + str(i + 1)
        teammate_cols[teammate_str] = []
    
    #fill in teammatenumber columnn
    for team_size in team_sizes:
        team_range = range(1, team_size + 1)
        for stu in team_range:
            teammatenumbers.append(stu)
     
    #fill in teamnumber column
    if not hasTeamNumbers:
        for team in teams:
            subset_section = raw[raw[teamNameCol] == team].reset_index()
            curr_team = ''
            counter = 0
            for i in range(len(subset_section)):
                if subset_section.iloc[i][teamNameCol] == curr_team:
                    teamnumbers.append(team + '_Team_' + str(counter))
                else:
                    counter = counter + 1
                    curr_team = subset_section.iloc[i][teamNameCol]
                    teamnumbers.append(team + '_Team_' + str(counter))
        raw["TeamNumber"] = teamnumbers
    
    #number columns set
    raw["TeammateNumber"] = teammatenumbers
    raw = raw.reset_index(drop=True)
    
    #fill in teammateX names
    for team in teams:
        subset_team = raw[raw[teamNameCol] == team]
        team_size = len(subset_team)
        counter = 1
        for teammate_col in list(teammate_cols.keys()):
            curr_stu = subset_team[subset_team["TeammateNumber"] == counter]
            for i in range(team_size):
                name_lst = curr_stu["FullName"].tolist()
                if len(name_lst) == 0:
                    teammate_cols[teammate_col].append('')
                else:
                    teammate_cols[teammate_col].append(name_lst[0])
            counter += 1
    
    for key in list(teammate_cols.keys()):
        raw[key] = teammate_cols[key]
        
    raw = raw.drop('FullNameUpper', axis=1)
        
    return raw.to_csv(outputFileName)


In [9]:
createRosterForQualtrics('SP24_E29_Roster_Raw.csv', 'First name', 'Last Name', 'group_name', True, 'Cleaned_SP24_E29_Roster_v2.csv')

In [10]:
#some light cleaning
roster = pd.read_csv("Cleaned_SP24_E29_Roster_v2.csv")
roster

,Unnamed: 0,First name,Last Name,group_name,Email address,Unnamed: 4,Unnamed: 5,Unnamed: 6,FullName,TeammateNumber,Teammate1,Teammate2,Teammate3,Teammate4
0,0,Allan,Zhong,Project Team 1,allanzhong@berkeley.edu,NaN,NaN,NaN,Allan Zhong,1,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira
1,1,Edwin,Lopez,Project Team 1,eclopez19@berkeley.edu,NaN,NaN,NaN,Edwin Lopez,2,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira
2,2,Jerrick,Wang,Project Team 1,jerrick@berkeley.edu,NaN,NaN,NaN,Jerrick Wang,3,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira
3,3,Simran,Kaur Khahira,Project Team 1,simrankk@berkeley.edu,NaN,NaN,NaN,Simran Kaur Khahira,4,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira
4,4,Miro,Garrett,Project Team 10,miroygarrett@berkeley.edu,NaN,NaN,NaN,Miro Garrett,1,Miro Garrett,Taiki Yamamoto,Taylor Jazan,Toren Andersen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,Siddhant,Vasudevan,Project Team 8,siddhant.vasudevan@berkeley.edu,NaN,NaN,NaN,Siddhant Vasudevan,4,Adrian Guan,Ayush Patel,Maithili Kumar,Siddhant Vasudevan
130,130,Allan,Ramirez,Project Team 9,allan.sram16@berkeley.edu,NaN,NaN,NaN,Allan Ramirez,1,Allan Ramirez,Arnav Patel,Owen Lachs,Tyler Peng
131,131,Arnav,Patel,Project Team 9,arnavp45@berkeley.edu,NaN,NaN,NaN,Arnav Patel,2,Allan Ramirez,Arnav Patel,Owen Lachs,Tyler Peng
132,132,Owen,Lachs,Project Team 9,owenlachs@berkeley.edu,NaN,NaN,NaN,Owen Lachs,3,Allan Ramirez,Arnav Patel,Owen Lachs,Tyler Peng


In [11]:
roster["TeamNumber"] = [teamName.split("Project")[1] for teamName in roster["group_name"]]

In [13]:
roster["Email"] = roster["Email address"]

In [14]:
roster

,Unnamed: 0,First name,Last Name,group_name,Email address,Unnamed: 4,Unnamed: 5,Unnamed: 6,FullName,TeammateNumber,Teammate1,Teammate2,Teammate3,Teammate4,TeamNumber,Email
0,0,Allan,Zhong,Project Team 1,allanzhong@berkeley.edu,NaN,NaN,NaN,Allan Zhong,1,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira,Team 1,allanzhong@berkeley.edu
1,1,Edwin,Lopez,Project Team 1,eclopez19@berkeley.edu,NaN,NaN,NaN,Edwin Lopez,2,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira,Team 1,eclopez19@berkeley.edu
2,2,Jerrick,Wang,Project Team 1,jerrick@berkeley.edu,NaN,NaN,NaN,Jerrick Wang,3,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira,Team 1,jerrick@berkeley.edu
3,3,Simran,Kaur Khahira,Project Team 1,simrankk@berkeley.edu,NaN,NaN,NaN,Simran Kaur Khahira,4,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira,Team 1,simrankk@berkeley.edu
4,4,Miro,Garrett,Project Team 10,miroygarrett@berkeley.edu,NaN,NaN,NaN,Miro Garrett,1,Miro Garrett,Taiki Yamamoto,Taylor Jazan,Toren Andersen,Team 10,miroygarrett@berkeley.edu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,Siddhant,Vasudevan,Project Team 8,siddhant.vasudevan@berkeley.edu,NaN,NaN,NaN,Siddhant Vasudevan,4,Adrian Guan,Ayush Patel,Maithili Kumar,Siddhant Vasudevan,Team 8,siddhant.vasudevan@berkeley.edu
130,130,Allan,Ramirez,Project Team 9,allan.sram16@berkeley.edu,NaN,NaN,NaN,Allan Ramirez,1,Allan Ramirez,Arnav Patel,Owen Lachs,Tyler Peng,Team 9,allan.sram16@berkeley.edu
131,131,Arnav,Patel,Project Team 9,arnavp45@berkeley.edu,NaN,NaN,NaN,Arnav Patel,2,Allan Ramirez,Arnav Patel,Owen Lachs,Tyler Peng,Team 9,arnavp45@berkeley.edu
132,132,Owen,Lachs,Project Team 9,owenlachs@berkeley.edu,NaN,NaN,NaN,Owen Lachs,3,Allan Ramirez,Arnav Patel,Owen Lachs,Tyler Peng,Team 9,owenlachs@berkeley.edu


In [16]:
roster[["FullName", "First name", "Last Name", "Email", "TeamNumber", "TeammateNumber", 'Teammate1', 'Teammate2', 'Teammate3', 'Teammate4']].to_csv("FINAL_E29_ROSTER.CSV")